In [0]:
import pandas as pd
import numpy as np
from functools import reduce

In [6]:
alunos = pd.read_csv('alunos_caxias_2018.csv')
alunos.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
0,71CCAB7D5BAE6603B3995CA2EB8F9534,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,34,2014,Cursando,Graduação,Masculino
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2018,Cursando,Graduação,Masculino
2,43ABA22A4C7DC2F6E69C71CE90D339A0,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2017,Matrícula trancada,Graduação,Masculino
3,10707CB293B32AD1BC23EB641765BCE7,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2014,Matrícula trancada,Graduação,Feminino
4,7C95F1E0E536524DC3281C4452030F72,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,54,2007,Matrícula trancada,Graduação,Feminino


In [0]:
faixas_etarias = [[17,18,19],
                 [20,21,22,23],
                 [24,25,26],
                 [27,28,29]]

nomes_colunas = ['17-19',
                '20-23',
                '24-26',
                '27-29']

situacao = (alunos.TP_SITUACAO == 'Cursando')
nivel = (alunos.TP_NIVEL_ACADEMICO == 'Graduação')
ano_ingresso = (alunos.ANO_INGRESSO == 2018)


dados_faixa_etaria = []

for faixa_etaria, nome_coluna in zip(faixas_etarias, nomes_colunas):
  idades = (alunos.NU_IDADE.isin(faixa_etaria))
  ingressantes = alunos.loc[situacao & nivel & ano_ingresso & idades, :].copy()
  contagem = ingressantes.CO_IES.value_counts()
  tabela = pd.DataFrame(zip(contagem.keys(), contagem.values), columns = ['IES', nome_coluna])
  dados_faixa_etaria.append(tabela)

In [14]:
siglas = np.load('dicionario_ies_siglas.npy', allow_pickle=True).item()
siglas

{'Centro Universitário Da Serra Gaúcha': 'FSG',
 'Centro Universitário Uniftec': 'FTEC',
 'Faculdade América Latina': 'América Latina',
 'Faculdade Anglo-Americano De Caxias Do Sul': 'Anglo-Americano',
 'Faculdade Angloamericano De Caxias Do Sul': 'Anglo-Americano',
 'Faculdade Anhanguera De Caxias Do Sul': 'Anhanguera',
 'Faculdade Da Serra Gaucha': 'FSG',
 'Faculdade Da Serra Gaúcha': 'FSG',
 'Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul': 'FTSG',
 'Faculdade De Tecnologia Tecbrasil': 'FTEC',
 'Faculdade Dos Imigrantes  Fai': 'FAI',
 'Faculdade Dos Imigrantes - Fai': 'FAI',
 'Faculdade Ideau De Caxias Do Sul': 'IDEAU',
 'Faculdade La Salle - Caxias': 'La Salle',
 'Faculdade Montserrat': 'América Latina',
 'Faculdade Murialdo': 'Murialdo',
 'Faculdade Nossa Senhora De Fatima': 'Fátima',
 'Faculdade Nossa Senhora De Fátima': 'Fátima',
 'Faculdade Serrana': 'IDEAU',
 'Universidade De Caxias Do Sul': 'UCS'}

In [17]:
ingressantes_por_faixa_etaria = reduce(lambda left, right: pd.merge(left, right, how = 'outer', on = 'IES'), dados_faixa_etaria)
ingressantes_por_faixa_etaria.IES.replace(siglas, inplace = True)
ingressantes_por_faixa_etaria

,IES,17-19,20-23,24-26,27-29
0,UCS,1234,926,308,175
1,FSG,925,1129,586,462
2,FTEC,256,412,272,255
3,Anhanguera,73,111,67,42
4,Fátima,23,24,22,8
5,Murialdo,21,33,10,12
6,FTSG,15,31,20,15
7,IDEAU,14,18,14,5


In [0]:
ingressantes_por_faixa_etaria.to_csv('ingressantes_por_faixa_etaria.csv', index = False)